# Rosenblatt perceptron and SVM

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from dataclasses import dataclass

In [ ]:
import matplotlib as mpl
mpl.rcParams['axes.grid'] = True  # Use per default a grid, i.e. plt.grid()
# mpl.rcParams['figure.figsize'] = [6.4, 4.8]  # Change the default figure size

In [ ]:
# numpy.set_printoptions:
#     threshold: Total number of array elements which trigger summarization rather than full repr (default 1000).
np.set_printoptions(threshold=100)

# Generate different datasets

In [ ]:
def map_0_1_to_minus_1_1(labels):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    """
    return 2 * labels - 1

In [ ]:
@dataclass
class Dataset:
    features: np.array
    labels: np.array

In [ ]:
def generate_circles_dataset(number_of_samples=1000, random_state=np.random):
    labels = random_state.choice([0, 1], size=number_of_samples)
    features = random_state.normal(size=(number_of_samples, 2))
    features /= np.linalg.norm(features, axis=-1, keepdims=True)
    features *= labels[:, None] + 1 / 2 * random_state.uniform(size=number_of_samples)[:, None]
    return Dataset(features, map_0_1_to_minus_1_1(labels))

def generate_blobs_dataset(number_of_samples=1000, scale=1, random_state=np.random):
    labels = random_state.choice([0, 1], size=number_of_samples)
    cluster_centers = random_state.normal(scale=scale, size=(2, 2))
    features = cluster_centers[labels, :] + random_state.normal(size=(number_of_samples, 2))
    return Dataset(features, map_0_1_to_minus_1_1(labels))

def generate_checkerboard_dataset(number_of_samples=1000, random_state=np.random):
    features = random_state.uniform(-1, 1, size=(number_of_samples, 2))
    labels = np.sign(np.prod(features, axis=1)).astype(np.int)
    return Dataset(features, labels)

In [ ]:
circles = generate_circles_dataset(random_state=np.random.RandomState(0))
easy_blobs = generate_blobs_dataset(scale=4, random_state=np.random.RandomState(0))
checkerboard = generate_checkerboard_dataset(random_state=np.random.RandomState(0))
hard_blobs = generate_blobs_dataset(scale=1, random_state=np.random.RandomState(0))

# Plot the different datasets

Which of the datasets are linearly separable?

In [ ]:
def plot_dataset(dataset, ax=None, title=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    classes = np.unique(dataset.labels)
    classes = np.sort(classes)
    for c in classes:
        ax.scatter(
            dataset.features[dataset.labels == c, 0],
            dataset.features[dataset.labels == c, 1],
            label=f'Class: {c}',
        )
    ax.set_aspect('equal')
    if title is not None:
        ax.set_title(title)
    ax.legend()

figure, axes = plt.subplots(2, 2, figsize=(12, 12))
plot_dataset(circles, ax=axes[0, 0], title='circles')
plot_dataset(easy_blobs, ax=axes[0, 1], title='easy_blobs')
plot_dataset(checkerboard, ax=axes[1, 0], title='checkerboard')
plot_dataset(hard_blobs, ax=axes[1, 1], title='hard_blobs')

# Rosenblatt Perceptron

For a shorter notation, lets use the following abbreviations:
\begin{align}
\tilde{\mathbf{w}} &= \begin{pmatrix}\mathbf w \\ w_0\end{pmatrix}, &
\tilde{\mathbf{x}}_n &= \begin{pmatrix}\mathbf x_n \\ 1\end{pmatrix}
\end{align}

Apply the Rosenblatt criterion to each of the datasets:
\begin{align}
J(\tilde{\mathbf{w}}) &= -\frac 1 N \sum_{n=1}^N \left(\frac{c_n - \hat c_n}{2}\right) \tilde{\mathbf{w}}^{\mathsf T} \tilde{\mathbf{x}}_n
\end{align}

Make sure to use some stopping criterion to avoid infinite loops. Plot the training loss over iterations. For which of the datasets does the training converge?

In [ ]:
def add_augmentation(x):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    
    Augmented vector: https://en.wikipedia.org/wiki/Affine_transformation#Augmented_matrix
    """
    return ???

def remove_augmentation(x_tilde):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    """
    return ???

In [ ]:
def predict_rosenblatt_perceptron(x_tilde, w_tilde, transform_fn=None):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    """
    if transform_fn is not None:
        x = remove_augmentation(x_tilde)
        x = transform_fn(x)
        x_tilde = add_augmentation(x)
    
    discriminant = ???
    prediction = ???
    return discriminant, prediction

In [ ]:
def fit_rosenblatt_perceptron(dataset, iterations=1000, learning_rate=0.1, transform_fn=None):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    """
    loss_history = np.empty(iterations)
    loss_history[:] = np.nan
    N = dataset.features.shape[0]
    
    if transform_fn is None:
        x_tilde = add_augmentation(dataset.features)
    else:
        x = transform_fn(dataset.features)
        x_tilde = add_augmentation(x)
    
    w_tilde = np.random.normal(size=x_tilde.shape[-1])
    c = dataset.labels
    for iteration in range(iterations):
        _, c_hat = predict_rosenblatt_perceptron(x_tilde, w_tilde)
        loss_history[iteration] = ???
        w_tilde = ???
    return w_tilde, loss_history

In [ ]:
def compute_complete_grid(xlim, ylim, steps):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    """
    x, y = np.meshgrid(
        np.linspace(*xlim, steps),
        np.linspace(*ylim, steps),
    )
    return x, y, np.stack([x, y], axis=-1)

def fit_and_plot(dataset, title, transform_fn=None, fit_function=fit_rosenblatt_perceptron, **kwargs):
    """
    ??? Add a doc string. What does this function do? What are the input parameters? Which shape?
    """
    w_tilde, loss_history = fit_function(
        dataset,
        transform_fn=transform_fn,
        **kwargs,
    )
    
    _, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].plot(loss_history)
    axes[0].set_title('{}, final loss: {}'.format(title, loss_history[-1]))
    # Use log scale above 1e-5 and linear scale below
    axes[0].set_yscale("symlog", linthreshy=1e-5)
    plot_dataset(dataset, ax=axes[1], title=title)
    
    # Plot decision boundary if in area
    steps = 100
    x, y, features_grid = compute_complete_grid(
        (np.min(dataset.features[:, 0]), np.max(dataset.features[:, 0])),
        (np.min(dataset.features[:, 1]), np.max(dataset.features[:, 1])),
        steps=steps
    )
    features_grid = features_grid.reshape(steps * steps, 2)
    features_grid = add_augmentation(features_grid)
    z, _ = predict_rosenblatt_perceptron(features_grid, w_tilde, transform_fn=transform_fn)
    z = np.reshape(z, (steps, steps))
    if np.any(z < 0) and np.any(z > 0):
        axes[1].contour(x, y, z, levels=[0])
    else:
        axes[1].contour(x, y, z)
    plt.show()
    
    print('w_tilde', w_tilde)

- Which datasets are linearly separable?
- Explain the loss curve for the non linearly separable datasets.
- Is the loss curve an indicator if the we can get a reasonable discriminant for non linearly separable datasets?

In [ ]:
fit_and_plot(circles, 'circles')
fit_and_plot(easy_blobs, 'easy_blobs')
fit_and_plot(checkerboard, 'checkerboard')
fit_and_plot(hard_blobs, 'hard_blobs')

# Feature transformation

Which hand-crafted feature transformation is necessary to render the `circles` and `checkerboard` datasets linearly separable? Train a Rosenblatt perceptron using this transformation. Plot the trainig loss over iterations.

- Analyse the decision boundarys.
- Which solution generalizes better?
- For which dataset do you expect a better solution from SVM?

In [ ]:
fit_and_plot(
    circles,
    'circles',
    transform_fn=lambda x: ???
)
fit_and_plot(
    checkerboard,
    'checkerboard',
    iterations=10000,
    transform_fn=lambda x: ???
)

# Support vector machine

Train an SVM using gradient descent. For which datasets does the algorithm converge?

\begin{align}
	\tilde{\mathcal{L}}(\mathbf{w},w_0) = \frac{\|\mathbf{w}\|^2}{2} + \frac C N \sum_{n=1}^N \max\left(0, 1 - c_n(\mathbf{w}^{\mathsf T}\mathbf{x}_n+w_0) \right).
\end{align}


Text from SML script to this equation (Eq. 4.36):

> The first term will push the model to have a small weight vector $\mathbf w$, leading to a large
margin, while the second term computes the total of all margin violations. Minimizing
this term ensures that the model makes the margin violations as small and as few as
possible. $C$ is a trade-off parameter between the two contributions to the cost function.
This objective function can be minimized by gradient descent.
>
> By the way, the function max(0, 1−x) is the hinge loss we have seen earlier in Section 2.7.3

Apply the same feature transform as before to render the `circles` and `checkerboard` dataset linearly separable.



The gradients are:
\begin{align}
\frac{\partial J}{\partial \mathbf w} &= \mathbf w + \frac C N \sum_{n=1}^N -c_n \mathrm{heaviside}\bigg(1 - c_n(\mathbf{w}^{\mathsf T}\mathbf{x}_n+w_0)\bigg) \mathbf x_n \\
\frac{\partial J}{\partial w_0} &= \frac C N \sum_{n=1}^N -c_n \mathrm{heaviside}\bigg(1 - c_n(\mathbf{w}^{\mathsf T}\mathbf{x}_n+w_0)\bigg)
\end{align}

In [ ]:
def fit_support_vector_machine(dataset, transform_fn=None, iterations=10000, learning_rate=0.01, hinge_loss_weight=10):
    loss_history = np.empty(iterations)
    loss_history[:] = np.nan
    N = dataset.features.shape[0]
    x_tilde = add_augmentation(dataset.features)
    
    if transform_fn is not None:
        x = remove_augmentation(x_tilde)
        x = transform_fn(x)
        x_tilde = add_augmentation(x)
    
    w_tilde = np.random.normal(size=x_tilde.shape[-1])
    c = dataset.labels
    for iteration in range(iterations):
        discriminant, c_hat = predict_rosenblatt_perceptron(x_tilde, w_tilde)
        loss_history[iteration] = ???
        gradient_w = ???
        gradient_w_0 = ???
        gradient = np.concatenate((gradient_w, [gradient_w_0]))
        w_tilde = w_tilde - learning_rate * gradient
    return w_tilde, loss_history

In [ ]:
fit_and_plot(easy_blobs, 'easy_blobs', fit_function=fit_support_vector_machine, hinge_loss_weight=1e4, learning_rate=0.001)
fit_and_plot(hard_blobs, 'hard_blobs', fit_function=fit_support_vector_machine)

In [ ]:
fit_and_plot(circles, 'circles', fit_function=fit_support_vector_machine, transform_fn=lambda x: np.linalg.norm(x, axis=-1, keepdims=True))
fit_and_plot(
    checkerboard, 'checkerboard',
    fit_function=fit_support_vector_machine,
    transform_fn=lambda x: ???,
    iterations=???,
    learning_rate=???,
    hinge_loss_weight=???,
)

In [ ]:
sign_dataset = Dataset(np.array([[1, 1], [-1, 1], [1, -1], [-1, -1]]) * 100, np.array([-1, -1, 1, 1]))
plot_dataset(sign_dataset)

- How should the decision boundary look like?
- Can you explain, why the SVM code does not work for this simple example?

In [ ]:
fit_and_plot(sign_dataset, 'sign', fit_function=fit_support_vector_machine)

In [ ]:
sign_dataset2 = Dataset(np.array([[1, 1], [-1, 1], [0, -1], [0, -1.5]]), np.array([-1, -1, 1, 1]))
plot_dataset(sign_dataset2)

- How should the decision boundary look like?
- Can you explain, why the decision boundary is not the expected one?

In [ ]:
fit_and_plot(sign_dataset2, 'sign2', fit_function=fit_support_vector_machine, hinge_loss_weight=1)

# For experts...

- The SVM parameters can be learned faster by using an appropriate solver instead of using gradient descent.
Check how the parameters are obtained in the `sklearn` source code and use it to verify your results.
- Mark the support vectors in your diagrams.
- What is the advantage of training the SVM with gradient descent?